<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Hands-on%20ML/chapter12/Customizing_Models_and_Training_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras

# **Custom Loss Functions**

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [4]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_error_small = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_error_small, squared_loss, linear_loss)

In [5]:
model.compile(loss=huber_fn, optimizer='nadam')

In [6]:
tf.keras.models.save_model(model, 'my_model_with_custom_loss.h5')

In [7]:
model = tf.keras.models.load_model('my_model_with_custom_loss.h5', 
                            custom_objects={'huber_fn': huber_fn})

# What if you want to add a threshold to measure whether the error is small or not? 

In [8]:
def create_hubor(threshold=1):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_error_small = tf.abs(error) < 1
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_error_small, squared_loss, linear_loss)
    return huber_fn


In [9]:
model.compile(loss=create_hubor(2.0), optimizer='nadam')

In [10]:
tf.keras.models.save_model(model, 'my_model_with_custom_loss_2.h5')

In [11]:
model = tf.keras.models.load_model('my_model_with_custom_loss_2.h5', 
                                   custom_objects={'huber_fn': create_hubor(2.0)})

# You have to pass the threshold whenever you load the model, so we can overcome this problem by subclassing keras.losses.Loss class

In [12]:
class HuborLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_error_small = tf.abs(error) < 1
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_error_small, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

In [13]:
model.compile(loss=HuborLoss(2.), optimizer='nadam')

In [14]:
keras.models.save_model(model, 'my_model_with_custom_loss_3.h5')

In [15]:
model = keras.models.load_model('my_model_with_custom_loss_3.h5', 
                                custom_objects={'HuborLoss': HuborLoss})

# **Custom Activation**

In [16]:
def my_softplus(z):
    return tf.log(tf.exp(z) + 1.)

# **Custom Initializer**

In [17]:
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / shape[0] + shape[1])
    return tf.random.normal(shape, stdev=stdev, dtype=dtype)

# Custom Regularizer

In [18]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# **Custom Constraint**

In [19]:
def my_postivie_weight(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [20]:
layer = keras.layers.Dense(30, 
                           activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_postivie_weight)

**If for a function we need to save the hyperparameter with the model, we will have to subclass it**

In [21]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        super().__init__()
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))

    def get_config(self):
        return {'factor': self.factor}

# **Custom Metrics**


In [22]:
class HuborMetric(keras.metrics.Metric):
    def __init__(self, threshold=1., **kwargs):
        super().__init(**kwargs)
        self.threshold = threshold
        self.hubor_fn = create_hubor(threshold)
        self.total = self.add_weight('total', initializer='zeros'),
        self.count = self.add_weight('count', initializer='zeros')
    
    def update_state(self, y_true, y_pred):
        metric = self.hubor_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.total.assign_add(tf.cast(tf.size(y_true)), dtype=tf.float32)

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}


# **Custom Layers**

In [23]:
my_exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [57]:
class MyDenseLayer(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        self.units = units
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=[batch_input_shape[-1], self.units],
                                      initializer='glorot_normal')
        self.bias = self.add_weight(name='bias', shape=[self.units], initializer='zeros')
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.tensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units': self.units,
                'activation': keras.activations.serialize(self.activation)}

In [58]:
model_with_custom_layers = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MyDenseLayer(300, activation='relu'),
    MyDenseLayer(100, activation='relu'),
    MyDenseLayer(10, activation='softmax')
])

model_with_custom_layers.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
my_dense_layer_9 (MyDenseLay (None, 300)               235500    
_________________________________________________________________
my_dense_layer_10 (MyDenseLa (None, 100)               30100     
_________________________________________________________________
my_dense_layer_11 (MyDenseLa (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [60]:
model_with_custom_layers.layers[1].get_config()

{'activation': 'relu',
 'dtype': 'float32',
 'name': 'my_dense_layer_9',
 'trainable': True,
 'units': 300}

In [56]:
model.layers[1].get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'dtype': 'float32',
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'kernel_regularizer': None,
 'name': 'dense',
 'trainable': True,
 'units': 300,
 'use_bias': True}